In [3]:
import os
import json
import pandas as pd
import traceback

In [4]:
from langchain.chat_models import ChatOpenAI

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
KEY=os.getenv("OPENAI_API_KEY")

In [10]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

c:\genmcq\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Importing all the necessary libraries 

In [11]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain 
from langchain.callbacks import get_openai_callback # get_openai_callback is used for tracking the number of tokens
import PyPDF2

In [12]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [13]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [17]:
# Creating object of the PromptTemplate class
quiz_gen_prompt = PromptTemplate(
    input_variables=["text","number", "subject","tone", "response_json"], # Variables user will pass in the prompt
    template = TEMPLATE     
)

Now I will create LLM Chain. LLM chain is used to connect multiple components together.
Here the 2 components are PromptTemplate and LLM.

In [22]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_gen_prompt, output_key="quiz", verbose=True)

In [19]:
# This prompt is regarding to the evaluation of the quiz
TEMPLATE_2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [20]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [21]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [23]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)

In [24]:
file_path="C:\genmcq\data.txt"

In [25]:
with open(file_path, "r") as f:
    text = f.read()

In [30]:
text

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [26]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON) 

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [27]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [29]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": text,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\genmcq\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [31]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1922
Prompt Tokens:1192
Completion Tokens:730
Total Cost:0.003248


In [32]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [35]:
quiz = response.get("quiz")

In [38]:
quiz = json.loads(quiz)

In [39]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [40]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a: Chemistry | b: Biology | c: Physics | d: Geology',
  'Correct': 'b'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Photosynthesis | c: Mitosis | d: Genetics',
  'Correct': 'a'},
 {'MCQ': 'What is the basic unit of life that all organisms are made up of?',
  'Choices': 'a: Atoms | b: Molecules | c: Cells | d: Tissues',
  'Correct': 'c'},
 {'MCQ': 'Which process allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Photosynthesis | b: Respiration | c: Digestion | d: Fermentation',
  'Correct': 'b'},
 {'MCQ': 'What is the age of life on Earth according to the text?',
  'Choices': 'a: 1 million years | b: 3.7 billion years | c: 100 million years | d: 10 billion years',
  'Correct': 'b'}]

In [41]:
quiz=pd.DataFrame(quiz_table_data)


In [42]:
quiz

,MCQ,Choices,Correct
0,What is the scientific study of life called?,a: Chemistry | b: Biology | c: Physics | d: Ge...,b
1,Which theme in biology explains the unity and ...,a: Evolution | b: Photosynthesis | c: Mitosis ...,a
2,What is the basic unit of life that all organi...,a: Atoms | b: Molecules | c: Cells | d: Tissues,c
3,"Which process allows organisms to move, grow, ...",a: Photosynthesis | b: Respiration | c: Digest...,b
4,What is the age of life on Earth according to ...,a: 1 million years | b: 3.7 billion years | c:...,b


In [43]:
quiz.to_csv("biology.csv",index=False)

In [44]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_03_2024_15_50_10'